In [14]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd

In [67]:

#This is live data from the our world in data github. This is the csv they put on the website. Covid19 data.
data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1 = data1.astype(str)
data1 = data1[data1 != 'None']
data1 = data1['total_deaths'].astype(float)


TypeError: 'Series' object is not callable